In [1]:
# import py_entitymatching package
import py_entitymatching as em

/Users/Hacker/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# specify filepaths for tables A and B. 
path_A = '/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage3/formattedTableA.csv' # path_A is the file path where table_A.csv is stored
path_B = '/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage3/formattedTableB.csv' # path_B is the file path where table_B.csv is stored

In [3]:
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A, key='id')

# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
# Perform Attribute equivalence blocking on publishedYear, publishedMonth, publishedDay, pages
# Essentially multi-attributes equivalence blocking in BlackBoxBlocker
bb = em.BlackBoxBlocker()

In [5]:
# Detect the inconsistent type issue of "publishedYear", perform the corresponding type conversion so er
A['publishedYear'] = A['publishedYear'].convert_objects(convert_numeric=True)

/Users/Hacker/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [6]:
# Define the black_box function used to check the equivalence between numerical attributes
def numeric_value_block(x, y):
    x_day = x['publishedDay']
    y_day = y['publishedDay']
    x_month = x['publishedMonth']
    y_month = y['publishedMonth']
    x_year = x['publishedYear']
    y_year = y['publishedYear']
    x_pages = x['pages']
    y_pages = y['pages']
    day_diff = abs(x_day-y_day)
    month_diff = abs(x_month - y_month)
    year_diff = abs(x_year - y_year)
    pages_diff = abs(x_pages - y_pages)
    if day_diff > 2 or month_diff > 2 or year_diff > 2 or pages_diff > 10:
        return True
    else:
        return False

In [7]:
# Pass the numeric_value_block to black box function
bb.set_black_box_function(numeric_value_block)
Cnumeric = bb.block_tables(A, B, l_output_attrs=['title', 'authors', 'publisher', 'ISBN13','publishedYear','publishedMonth','pages'], r_output_attrs=['title', 'authors', 'publisher', 'ISBN13','publishedYear','publishedMonth','pages'], l_output_prefix='ltable_', r_output_prefix='rtable_', verbose=False, show_progress=True, n_jobs=4)

0%                          100%
[#                             ] | ETA: 00:12:29

KeyboardInterrupt: 

In [8]:
# Checking the output of the numeric blocker
Cnumeric

NameError: name 'Cnumeric' is not defined

In [ ]:
def isbn_partial_block(x,y):
    try:
        isbn_A = str(int(x['ISBN13']))
    except Exception:
        isbn_A = "0000000000000"
  
    isbn3_head_A = isbn_A[0:4]
    
    try:
        isbn_B = str(int(y['ISBN13']))
    except Exception:
        isbn_B = "0000000000000"
    
    isbn3_head_B = isbn_B[0:4]
    
    if(isbn3_head_A != isbn3_head_B):
        return True
    else:
        return False
    

In [ ]:
bb.set_black_box_function(isbn_partial_block)
Cisbn = bb.block_tables(A, B, l_output_attrs=['title', 'authors', 'publisher', 'ISBN13','publishedYear', 'publishedMonth', 'pages'], r_output_attrs=['title', 'authors', 'publisher', 'ISBN13', 'publishedYear', 'publishedMonth','pages'], l_output_prefix='ltable_', r_output_prefix='rtable_', verbose=True, show_progress=True, n_jobs=4)

In [ ]:
Cisbn

In [ ]:
C = em.combine_blocker_outputs_via_union([Cnumeric, Cisbn])

In [ ]:
C

In [ ]:
feature_table = em.get_features_for_blocking(A,B)

In [ ]:
feature_table

In [ ]:
title_blocker=em.RuleBasedBlocker()

In [ ]:
title_blocker.add_rule(['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5'], feature_table)

In [ ]:
Ctitle = title_blocker.block_candset(C, verbose=False, show_progress=True, n_jobs=4)

In [ ]:
Ctitle

In [ ]:
# save the table C to csv file
em.to_csv_metadata(Ctitle, './tableC.csv')

In [ ]:
# specify the file path for table A and table B
path_A = "/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage4/formattedTableA.csv"
path_B = "/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage4/formattedTableB.csv"

In [ ]:
# specify the file path for table C got from blocking stage
path_C = "/Users/Hacker/Documents/Courses/Fall 2016/CS638/projects/CS638/stage4/tableC.csv"

In [ ]:
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [ ]:
# read table C
C = em.read_csv_metadata(path_C, ltable=A, rtable=B)

In [ ]:
sample_table = em.sample_table(C,400,replace=False,verbose=False)

In [ ]:
# sample 400 tuple pairs from table C
sample_table

In [ ]:
# label the sample tuple pairs
gold_table = em.label_table(sample_table,'gold',verbose=False)

In [ ]:
# save the tableG to csv file
em.to_csv_metadata(gold_table, './tableG.csv')